# 생물정보학 및 실습 1 - Free Project
## LIN28A binding motif predictor 만들기 

### 환경 세팅 하기 

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


 Bioconda 환경 설치


In [2]:
!git clone https://github.com/hyeshik/colab-biolab.git
!cd colab-biolab && bash tools/setup.sh
exec(open('colab-biolab/tools/activate_conda.py').read())

Cloning into 'colab-biolab'...
remote: Enumerating objects: 69, done.
remote: Counting objects: 100% (69/69), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 69 (delta 22), reused 54 (delta 13), pack-reused 0
Unpacking objects: 100% (69/69), done.
./
./root/
./root/.bin.priority/
./root/.bin.priority/pip
./root/.bin.priority/pip2
./root/.bin.priority/pip3
./root/.tmux.conf
./root/.vimrc
./root/.profile
./root/.bashrc.biolab
./root/.condarc
--2022-06-08 08:13:02--  https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8303, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 76607678 (73M) [application/x-sh]
Saving to: ‘miniconda3.sh’

miniconda3.sh       100%[===================>]  73.06M   272MB/s    in 0.3s    

2022-06-08 08:13:02 (272 MB/s) - ‘miniconda3.sh’ save

In [3]:
!conda install -y bedtools bioawk

Solving environment: \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / done


==> WARNING: A newer version of conda exists. <==
  current version: 4.12.0
  latest version: 4.13.0

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /root/conda/envs/lab

  added / updated specs:
    - bedtools
    - bioawk


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    bioawk-1.0                 |       h7132678_7         196 KB  bioconda
    ------------------------------------------------------------
                                           Total:         196 KB

The following NEW packages will be INSTALLED:

  bioawk             bioconda/linux-64::bioawk-1.0-h7132678_7



bioawk-1.0           | 196 KB    | : 100% 1.0/1 [00:00<00:00, 10.16it/s]
Preparing transaction: \ 

In [4]:
%cd /content/drive/MyDrive/binfo1-datapack01/

/content/drive/MyDrive/binfo1-datapack01


### Fig SC3 그리기 

전체 Genome에 대해서 pileup 해주기 

In [ ]:
!samtools mpileup CLIP-35L33G.bam > CLIP-35L33G.pileup
!wc -l CLIP-35L33G.pileup

[mpileup] 1 samples in 1 input files
959697300 CLIP-35L33G.pileup


In [ ]:
!head CLIP-35L33G.pileup

chr1	3056473	N	4	^!G^!G^!G^!G	HGGI
chr1	3056474	N	4	AAAA	HB@I
chr1	3056475	N	4	AAAA	GG?I
chr1	3056476	N	4	TTTT	HGGH
chr1	3056477	N	4	GGGG	HEGI
chr1	3056478	N	4	GGGG	GEEI
chr1	3056479	N	4	AAAA	EA=I
chr1	3056480	N	4	AAAA	@AEH
chr1	3056481	N	4	GGGG	GFFI
chr1	3056482	N	4	TTTT	G=BG


In [ ]:
!tail CLIP-35L33G.pileup

MU069435.1	31102	N	4	GGGG	IDDB
MU069435.1	31103	N	4	GGGG	DEBE
MU069435.1	31104	N	4	GGGG	EDDE
MU069435.1	31105	N	4	TTTT	@><8
MU069435.1	31106	N	4	TTT$T$	DG<;
MU069435.1	31107	N	2	GG	BC
MU069435.1	31108	N	2	GG	B>
MU069435.1	31109	N	2	TT	@3
MU069435.1	31110	N	2	GG	B1
MU069435.1	31111	N	2	G$G$	@6


Filtering with depth (30)

In [ ]:
!awk '$4 > 30 {print $0;}' CLIP-35L33G.pileup > CLIP-35L33G-filt.pileup
!wc -l CLIP-35L33G-filt.pileup

45781030 CLIP-35L33G-filt.pileup


In [ ]:
!tail CLIP-35L33G-filt.pileup

MU069435.1	1559	N	346	>>>>TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT$TT$T$T$TTTTTTTTTTTTTTT$TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT	DIDHGHIIIIIIEEIEIGIIEGGBGGDG:BEBHDDFBIGIEHGIIGIIIHBGIBB>G?DI3IGBEI7GGGGI4GII=G?IIGDIIHIIEHGIHIHGEIHIG=EIHGIIHIGIIIIDIDIIIEIIIIIIGIHIIFI@GGIIIIBIIEIGHH?@IAIGI@HIBHHHFAGIGHGHDIIGIGIIBGIEFHIGGIIIIGHIHIDIIFIHIGGIH@DHGHHDIEDIIEIII8GDHHGHIBGIIHIDIAIGIHIIHEG4II>HD?IH@IFIHGIGDHH;AFGH@EGH:BCHG>IEIDIHIHIIHIIIIIIHHGHIGIIGIGFHGCII=HD@HI?IIIGHGGIHIGIDDH>HFB
MU069435.1	1560	N	341	>>>>TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT$T$T$T$TT$T$TT$T$T$T$TT$TT$TT$TTT$TT$T$T$TT$TT$T$TTT$T$TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT

In [13]:
!awk '$1 == "chr1" {print $0;}' CLIP-35L33G-filt.pileup > CLIP-33L33G-filt-chr1.pileup

In [15]:
!head CLIP-33L33G-filt-chr1.pileup

chr1	3221903	N	33	AAAAAAAAAAAA^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A^!A	ID@@DEG@@IGHEHHDIIIIIGGGBIFGHD@:H
chr1	3221904	N	33	TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT	BDAGDAD;@AHH2HH7IIIIIGHHBIDDGBG@G
chr1	3221905	N	33	TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT	IDAFDDG@@AHD2HEBIIIIEGHH@IEGHGDE?
chr1	3221906	N	33	TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT	AFDH>ED@@AHHDHHDIIIIGGHH@HCEHDBEH
chr1	3221907	N	33	TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT	IFDH@DH@@AFD3HH3IIIIGGGH@HFGHGGEH
chr1	3221908	N	33	AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA	IH@A@<G@=IHH:HHBIIIIIBGHHI@DHBE@G
chr1	3221909	N	33	TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT	HHDH@<G@=FHH=HH<IIIIIGHHCIDAH@9DD
chr1	3221910	N	33	GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG	IHDH>8D==IHH=HH3IIIIIGHHIIDAHD;DH
chr1	3221911	N	33	AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA	IFDB<BB<BIFB:HHBIIIIIGGHHIGGGG?BH
chr1	3221912	N	33	TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT	IHDGDBGDDIHHBEH:IIHIIGHHEIDBHGEBH


Baseread에서 Match와 Substitution만 남기기 

In [5]:
import pandas as pd 
import re 
import collections 
import math

def estimate_shannon_entropy(rna_sequence):
    m = len(rna_sequence)
    bases = collections.Counter([tmp_base for tmp_base in rna_sequence])
 
    shannon_entropy_value = 0
    for base in bases:
        # number of residues
        n_i = bases[base]
        # n_i (# residues type i) / M (# residues in column)
        p_i = n_i / float(m)
        entropy_i = p_i * (math.log(p_i, 2))
        shannon_entropy_value += entropy_i
 
    return shannon_entropy_value * -1


def chunk_processing(chunck): # define a function that you will use on chunks
    toremove = re.compile('-[0-9]+[ACGTNacgtn]+')
    chunck['matches'] = chunck['basereads'].apply(lambda x: toremove.sub('', x))
    toremove = re.compile('\+[0-9]+[ACGTNacgtn]+')
    chunck['matches'] = chunck['matches'].apply(lambda x: toremove.sub('', x))
    toremove = re.compile('[<>$*#]')
    chunck['matches'] = chunck['matches'].apply(lambda x: toremove.sub('', x))
    toremove = re.compile('[\^][\w]')
    chunck['matches'] = chunck['matches'].apply(lambda x: toremove.sub('', x))
    toremove = re.compile('[\^][\W]')
    chunck['matches'] = chunck['matches'].apply(lambda x: toremove.sub('', x))
    chunck['entropy'] = chunck.apply(lambda x: estimate_shannon_entropy(x['matches']), axis=1)
    return chunck


chunk_list = [] # create an empty list to hold chunks
chunksize = 10 ** 6 # set chunk size
for chunk in pd.read_csv('CLIP-35L33G-filt.pileup', sep='\t', names=['chrom', 'pos', '_ref', 'count', 'basereads', 'quals'],chunksize=chunksize): # read in csv in chunks of chunksize
    processed_chunk = chunk_processing(chunk) # process the chunks with chunk_processing() function
    chunk_list.append(processed_chunk) # append the chunks to a list
df_concat = pd.concat(chunk_list) # concatenate the list to a dataframe
df_concat.to_csv("CLIP-35L33G-filt.csv", sep='\t', index=False)

In [5]:
import pandas as pd 
import re 
import collections 
import math

In [6]:
df_concat = pd.read_csv("CLIP-35L33G-filt.csv", sep='\t')

Filtering with Entropy (0.7) 

In [9]:
filt_entropy = df_concat[(df_concat['entropy'] >= 1.0) & (df_concat['count'] > 60)].copy()
# filt_entropy.to_csv("CLIP-35L33G-filt_entropy.csv", sep='\t', index=False, columns=['chrom', 'pos', '_ref', 'count', 'basereads', 'quals', 'matches', 'entropy'], )
# filt_entropy = pd.read_csv("CLIP-35L33G-filt_entropy.csv", sep='\t', names=['chrom', 'pos', '_ref', 'count', 'basereads', 'quals', 'entropy'])
th_filt_entropy = filt_entropy.sort_values(by='entropy', ascending=False).nlargest(1000, 'entropy')

In [8]:
th_filt_entropy

,chrom,pos,_ref,count,basereads,quals,matches,entropy
45759046,chrY,90803210,N,73,>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>><<<<<<<<<<<<<...,IIIIEIFDE@GIGI>IIIDIIIG9=I9GIH7CGEIIIHIAIIAGIG...,ccTTTTTCCtttttcccccccaaa,2.147317
45683317,chrX,168772710,N,168,<ccccccccccctcccccttcctcccccccCCCCCccccccccccT...,G?1IHIGH<DHI@HIDGDGFBGBGHHIHEII3IEGGGHIIGGEIGG...,ccccccccccctcccccttcctcccccccCCCCCccccccccccTT...,2.107874
19109999,chr7,75422105,N,62,<<<<<ggggggggggtattctttctctttggggggtcttttttttc...,EHIIGIIIGIDHEIHIEHDHGIHGGHI@GHHBG=GG?EI>H@EIBI...,ggggggggggtattctttctctttggggggtcttttttttcccccT...,2.043703
30252701,chr11,108902487,N,1046,>>>>>>><<TCTCGTAGTGTGCTCCCACGCCTTTTTGGCTTAATTC...,BIHIGD?I4H@<HHBH9@BFEDGGEHIIIIHCIIDGGH3F7FHI;E...,TCTCGTAGTGTGCTCCCACGCCTTTTTGGCTTAATTCCGTACGGTT...,2.001411
18304352,chr7,14350331,N,187,gggggggggggCCCCTCTTCAGC**CGGCCAG**TTT****CTTCC...,EHGHIGI:GDD@GGI>H<III@CEDHIBHIIHIIIHI8HDIHHIGD...,gggggggggggCCCCTCTTCAGCCGGCCAGTTTCTTCCCTCTCCTC...,1.997098
...,...,...,...,...,...,...,...,...
39531319,chr16,84620886,N,488,>>>CGGGGGGAAGAAGCGGAAGGAGGCCGAGCAAGGAGGGACGGAG...,HGG6G:BIDIGHBIIHDDHGID7IIIDIBGGIGIHGHHIIH><GEG...,CGGGGGGAAGAAGCGGAAGGAGGCCGAGCAAGGAGGGACGGAGGGC...,1.627179
18291015,chr7,11601250,N,199,CAGGACGCAGGCAA*GG-2NNCACACAG-2NNAG*G-2NNAGCGGG...,DI;H;IC;D?HIHEG?IHGDGHF>IGIDCEIIHGDIHCHI0DCIIE...,CAGGACGCAGGCAAGGGCAGACCCGCCGCCAGAGAAGAAAGGAAAA...,1.627013
9064642,chr3,89077377,N,95,CCCCCCGGCGGGCCGGCGCTCAGGCCGTGGTCGAGGGAGGGGACGC...,FG?G?A?II@GFHIIGIFIHGGHIHGHIGHIIHDGHGHI5I@IH3I...,CCCCCCGGCGGGCCGGCGCTCAGGCCGTGGTCGAGGGAGGGGACGC...,1.626957
30186169,chr11,106964921,N,264,<tc-2nncccctgtcccttgcggccccgtcttcctc-2nngcgtc-...,/FII@IHIIHH<B>HBGAI?GIIGEGIDG>I<HIHIGIG<>IBIHE...,tcctcttccgcttactgctacgcttgtgccccgccccccc,1.626873


sorting 결과에서 앞뒤로 각 10nt씩 21nt단위로 fasta file 만들기 


(-)strand에 cross-linked된 read를 고려하기위해 annotation file load

In [10]:
!pip install pyranges

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 684 kB 27.3 MB/s 
     |████████████████████████████████| 470 kB 38.5 MB/s 
     |████████████████████████████████| 1.3 MB 47.9 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 425 kB 50.1 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for pyranges: filename=pyranges-0.0.117-py3-none-any.whl size=693480 sha256=7c99c983ad57644e2e4837479de8770bac6ac37c46d5cad7efaa545334ed158e
  Stored in directory: /root/.cache/pip/wheels/1e/c2/29/66f7e4c0de384a3f525410e00111fb88afeb8fbdde3064526f
  Created wheel for ncls: filename=ncls-0.0.65-cp37-cp37m-linux_x86_64.whl size=1750195 sha256=2903896b5c9b6f530f54c8730305e033ab9cceef5a970ca7b448c07a8b

In [11]:
import pyranges as pr

# as PyRanges-object
# annotation file을 pandas dataframe형태로 변환한다. 
gr = pr.read_gtf("gencode.gtf")
gencode = gr.df

In [25]:
gencode

,Chromosome,Source,Feature,Start,End,Score,Strand,Frame,gene_id,gene_type,...,transcript_type,transcript_name,transcript_support_level,tag,havana_transcript,exon_number,exon_id,protein_id,ccdsid,ont
0,chr1,HAVANA,gene,3143475,3144545,.,+,.,ENSMUSG00000102693.2,TEC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,chr1,HAVANA,transcript,3143475,3144545,.,+,.,ENSMUSG00000102693.2,TEC,...,TEC,4933401J01Rik-201,NA,basic,OTTMUST00000127109.1,NaN,NaN,NaN,NaN,NaN
2,chr1,HAVANA,exon,3143475,3144545,.,+,.,ENSMUSG00000102693.2,TEC,...,TEC,4933401J01Rik-201,NA,basic,OTTMUST00000127109.1,1,ENSMUSE00001343744.2,NaN,NaN,NaN
3,chr1,ENSEMBL,gene,3172238,3172348,.,+,.,ENSMUSG00000064842.3,snRNA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,chr1,ENSEMBL,transcript,3172238,3172348,.,+,.,ENSMUSG00000064842.3,snRNA,...,snRNA,Gm26206-201,NA,basic,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1869102,chrY,ENSEMBL,exon,90850137,90850446,.,-,.,ENSMUSG00000096850.2,protein_coding,...,protein_coding,Gm21748-201,NA,appris_principal_1,NaN,1,ENSMUSE00001062768.2,ENSMUSP00000137361.2,NaN,NaN
1869103,chrY,ENSEMBL,CDS,90850140,90850446,.,-,0,ENSMUSG00000096850.2,protein_coding,...,protein_coding,Gm21748-201,NA,appris_principal_1,NaN,1,ENSMUSE00001062768.2,ENSMUSP00000137361.2,NaN,NaN
1869104,chrY,ENSEMBL,start_codon,90850443,90850446,.,-,0,ENSMUSG00000096850.2,protein_coding,...,protein_coding,Gm21748-201,NA,appris_principal_1,NaN,1,ENSMUSE00001062768.2,ENSMUSP00000137361.2,NaN,NaN
1869105,chrY,ENSEMBL,stop_codon,90850137,90850140,.,-,0,ENSMUSG00000096850.2,protein_coding,...,protein_coding,Gm21748-201,NA,appris_principal_1,NaN,1,ENSMUSE00001062768.2,ENSMUSP00000137361.2,NaN,NaN


In [12]:
 # gencode에서 position이 포함되는 gene의 Strand정보를 가져온다. 
 
 def range_func(x):
  try:
    return gencode[(gencode['Start'] <= x) & (gencode['End'] >= x)]['Strand'].iloc[0]
  except:
    return None

 th_filt_entropy['start'] = th_filt_entropy['pos'] -11
 th_filt_entropy['end'] = th_filt_entropy['pos'] + 10 
 th_filt_entropy['strand'] = th_filt_entropy['pos'].apply(lambda x: range_func(x))

In [16]:
# null 제거 
th_filt_entropy = th_filt_entropy[th_filt_entropy['strand'].notnull()]

In [17]:
th_filt_entropy

,chrom,pos,_ref,count,basereads,quals,matches,entropy,start,end,strand
45759046,chrY,90803210,N,73,>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>><<<<<<<<<<<<<...,IIIIEIFDE@GIGI>IIIDIIIG9=I9GIH7CGEIIIHIAIIAGIG...,ccTTTTTCCtttttcccccccaaa,2.147317,90803199,90803220,+
45683317,chrX,168772710,N,168,<ccccccccccctcccccttcctcccccccCCCCCccccccccccT...,G?1IHIGH<DHI@HIDGDGFBGBGHHIHEII3IEGGGHIIGGEIGG...,ccccccccccctcccccttcctcccccccCCCCCccccccccccTT...,2.107874,168772699,168772720,-
19109999,chr7,75422105,N,62,<<<<<ggggggggggtattctttctctttggggggtcttttttttc...,EHIIGIIIGIDHEIHIEHDHGIHGGHI@GHHBG=GG?EI>H@EIBI...,ggggggggggtattctttctctttggggggtcttttttttcccccT...,2.043703,75422094,75422115,-
30252701,chr11,108902487,N,1046,>>>>>>><<TCTCGTAGTGTGCTCCCACGCCTTTTTGGCTTAATTC...,BIHIGD?I4H@<HHBH9@BFEDGGEHIIIIHCIIDGGH3F7FHI;E...,TCTCGTAGTGTGCTCCCACGCCTTTTTGGCTTAATTCCGTACGGTT...,2.001411,108902476,108902497,-
18304352,chr7,14350331,N,187,gggggggggggCCCCTCTTCAGC**CGGCCAG**TTT****CTTCC...,EHGHIGI:GDD@GGI>H<III@CEDHIBHIIHIIIHI8HDIHHIGD...,gggggggggggCCCCTCTTCAGCCGGCCAGTTTCTTCCCTCTCCTC...,1.997098,14350320,14350341,-
...,...,...,...,...,...,...,...,...,...,...,...
39531319,chr16,84620886,N,488,>>>CGGGGGGAAGAAGCGGAAGGAGGCCGAGCAAGGAGGGACGGAG...,HGG6G:BIDIGHBIIHDDHGID7IIIDIBGGIGIHGHHIIH><GEG...,CGGGGGGAAGAAGCGGAAGGAGGCCGAGCAAGGAGGGACGGAGGGC...,1.627179,84620875,84620896,-
18291015,chr7,11601250,N,199,CAGGACGCAGGCAA*GG-2NNCACACAG-2NNAG*G-2NNAGCGGG...,DI;H;IC;D?HIHEG?IHGDGHF>IGIDCEIIHGDIHCHI0DCIIE...,CAGGACGCAGGCAAGGGCAGACCCGCCGCCAGAGAAGAAAGGAAAA...,1.627013,11601239,11601260,+
9064642,chr3,89077377,N,95,CCCCCCGGCGGGCCGGCGCTCAGGCCGTGGTCGAGGGAGGGGACGC...,FG?G?A?II@GFHIIGIFIHGGHIHGHIGHIIHDGHGHI5I@IH3I...,CCCCCCGGCGGGCCGGCGCTCAGGCCGTGGTCGAGGGAGGGGACGC...,1.626957,89077366,89077387,+
30186169,chr11,106964921,N,264,<tc-2nncccctgtcccttgcggccccgtcttcctc-2nngcgtc-...,/FII@IHIIHH<B>HBGAI?GIIGEGIDG>I<HIHIGIG<>IBIHE...,tcctcttccgcttactgctacgcttgtgccccgccccccc,1.626873,106964910,106964931,-


Download genome reference

In [17]:
!wget https://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_mouse/release_M29/GRCm39.primary_assembly.genome.fa.gz --no-check-certificate

--2022-06-07 09:09:42--  https://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_mouse/release_M29/GRCm39.primary_assembly.genome.fa.gz
Resolving ftp.ebi.ac.uk (ftp.ebi.ac.uk)... 193.62.193.138
Connecting to ftp.ebi.ac.uk (ftp.ebi.ac.uk)|193.62.193.138|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 773873008 (738M) [application/octet-stream]
Saving to: ‘GRCm39.primary_assembly.genome.fa.gz’

GRCm39.primary_asse  43%[=======>            ] 319.87M   388KB/s    in 12m 56s 

2022-06-07 09:22:39 (422 KB/s) - Connection closed at byte 335403944. Retrying.

--2022-06-07 09:22:40--  (try: 2)  https://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_mouse/release_M29/GRCm39.primary_assembly.genome.fa.gz
Connecting to ftp.ebi.ac.uk (ftp.ebi.ac.uk)|193.62.193.138|:443... connected.
HTTP request sent, awaiting response... 206 Partial Content
Length: 773873008 (738M), 438469064 (418M) remaining [application/octet-stream]
Saving to: ‘GRCm39.primary_assembly.genome.fa.gz’

GRCm39.pr

In [19]:
!pip install Bio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 270 kB 16.9 MB/s 
     |████████████████████████████████| 2.3 MB 44.8 MB/s 


In [ ]:
from Bio import SeqIO
import gzip 

with gzip.open("GRCm39.primary_assembly.genome.fa.gz", "rt") as handle:
  seqs = SeqIO.parse(handle, "fasta")
  with open("xlinked_acr1.fasta", "a") as xh:
    for seq in seqs:
      for idx, row in th_filt_entropy.iterrows():
        ## position - 10 to position + 10 
        if seq.id == row['chrom']:
          xseq = seq.seq[row['start']:row['end']]
          print("plus" , str(xseq)[10])
          ## (-)strand는 reverse complement seq를 가져온다
          if row['strand'] == '-':
            xseq = xseq.reverse_complement()
            print("mius", str(xseq)[10])
          xseq = xseq.transcribe()
          # xh.write(str(xseq))
          # xh.write("\n")
  xh.close()


In [ ]:
with open("xlinked_reverse.fasta", "r") as handle:
  lines = handle.readlines()
  for line in lines:
    

WebLogo Result 

hexamer 추출하기 

In [ ]:
from Bio import SeqIO
import gzip 

with gzip.open("GRCm39.primary_assembly.genome.fa.gz", "rt") as handle:
  seqs = SeqIO.parse(handle, "fasta")
  with open("trainset.txt", "a") as xh:
    for seq in seqs:
      for poskey, posval in pos_dict_.items():
        if seq.id == poskey:
          for v in posval: 
            print(v)
            print(seq.seq[int(v)-10:int(v)+10])
            xh.write(str(seq.seq[int(v)-2:int(v)+4]))
            xh.write("\n")
            ## position - 10 to position + 10 
  xh.close()

In [ ]:
len(filt_entropy)

487484

In [ ]:
fif_filt_entropy = filt_entropy.nlargest(50000, 'entropy').copy()

In [ ]:
under_entropy = df_concat[df_concat['entropy'] == 0].sample(50000).copy()

In [ ]:
len(under_entropy)

50000

In [ ]:
under_entropy.to_csv("CLIP-35L33G-50-under_entropy.csv", sep='\t', index=False, columns=['chrom', 'pos', '_ref', 'count', 'basereads', 'quals', 'entropy'])
# fif_filt_entropy.to_csv("CLIP-35L33G-50-over_entropy.csv", sep='\t', index=False, columns=['chrom', 'pos', '_ref', 'count', 'basereads', 'quals', 'entropy'])

In [ ]:
fif_filt_entropy['start'] = fif_filt_entropy['pos'] - 2
fif_filt_entropy['end'] = fif_filt_entropy['pos'] + 4
fif_filt_entropy.to_csv('50-over_entropy.bed', sep='\t', index=False, header=False, columns=['chrom', 'start', 'end', 'entropy']) 

In [ ]:
under_entropy['start'] = under_entropy['pos'] - 2
under_entropy['end'] = under_entropy['pos'] + 4
under_entropy.to_csv('50-under_entropy.bed', sep='\t', index=False, header=False, columns=['chrom', 'start', 'end', 'entropy']) 

In [ ]:
!bedtools getfasta -fi GRCm39.primary_assembly.genome.fa -bed fifty-over_entropy.bed -fo fifty-over_entropy.fasta
!bedtools getfasta -fi GRCm39.primary_assembly.genome.fa -bed fifty-under_entropy.bed -fo fifty-under_entropy.fasta

In [ ]:
!gzip -d GRCm39.primary_assembly.genome.fa.gz

Training Model - CNN 